# Importación de las librerias

In [71]:
import pandas as pd
import numpy as np

# Leer los datos

In [72]:
datos = pd.read_csv("REG02_EPHC_ANUAL_2024.csv", sep=";")

/tmp/ipykernel_12036/3740968167.py:1: DtypeWarning: Columns (197,199,202,206,207,208,209,210,211) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("REG02_EPHC_ANUAL_2024.csv", sep=";")


In [73]:
datos.head()

,UPM,NVIVI,NHOGA,AÑO,DPTO,AREA,L02,P02,P03,P04,...,ingrevasode,ingrealmuerzode,ipcm,pobrezai,pobnopoi,quintili,decili,quintiai,decilai,informalidad
0,14,3,1,2024,0,1,2,40,1,1,...,0,0,"4035459,49658286",3,0,5,10,5,9,2
1,14,3,1,2024,0,1,6,38,2,1,...,0,0,"4035459,49658286",3,0,5,10,5,9,1
2,14,15,1,2024,0,1,1,78,1,1,...,0,0,"1782061,06870229",3,0,4,7,3,6,
3,14,15,1,2024,0,1,2,77,2,1,...,0,0,"1782061,06870229",3,0,4,7,3,6,
4,14,23,1,2024,0,1,1,32,1,1,...,0,0,"5931717,5572519",3,0,5,10,5,10,1


In [74]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57744 entries, 0 to 57743
Columns: 213 entries, UPM to informalidad
dtypes: int64(25), object(188)
memory usage: 93.8+ MB


# Filtrado de datos

Para el analisis voy a utilizar solo las siguientes columnas

- P06: Sexo
  - 1: Hombre
  - 6: Mujer
- P02: Edad
- PEAA: Ocupación
  - 1: Ocupado
  - 2: Desocupado
  - 3: Inactivo
- E01A: Ingreso mensual Ocupación principal declarado

In [75]:
# Dataframe con solo las columnas necesarias
datos_filtrados = datos[["P06", "P02", "PEAA", "E01A"]]

In [76]:
datos_filtrados.head()

,P06,P02,PEAA,E01A
0,1,40,1,3220000
1,6,38,1,3730373
2,1,78,3,0
3,6,77,3,0
4,1,32,1,5500000


In [77]:
datos_filtrados.shape

(57744, 4)

In [78]:
datos_filtrados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57744 entries, 0 to 57743
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   P06     57744 non-null  int64 
 1   P02     57744 non-null  int64 
 2   PEAA    57744 non-null  object
 3   E01A    57744 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


## Trabajo sobre los datos

In [79]:
# Renombrar las columnas
datos_filtrados.columns = ["Sexo", "Edad", "Ocupacion", "Ingreso"]

In [80]:
# Convertir la columna Ocupacion a int64

# Reemplazar el ' ' por NaN
datos_filtrados["Ocupacion"] = datos_filtrados["Ocupacion"].replace(" ", np.nan)

# Convertir a número
datos_filtrados["Ocupacion"] = pd.to_numeric(datos_filtrados["Ocupacion"], errors="coerce")

# Eliminar filas con NaN en Ocupacion
datos_filtrados = datos_filtrados.dropna(subset=["Ocupacion"])

# Pasar de float64 a int64
datos_filtrados["Ocupacion"] = datos_filtrados["Ocupacion"].astype("int64")

/tmp/ipykernel_12036/857654796.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_filtrados["Ocupacion"] = datos_filtrados["Ocupacion"].replace(" ", np.nan)
/tmp/ipykernel_12036/857654796.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_filtrados["Ocupacion"] = pd.to_numeric(datos_filtrados["Ocupacion"], errors="coerce")


In [82]:
# Seleccionar solo las personas ocupadas, es decir, Ocupacion = 1
datos_filtrados = datos_filtrados[datos_filtrados["Ocupacion"] == 1]

In [83]:
# Ahora solo hay personas ocupas
datos_filtrados["Ocupacion"].unique()

array([1])

In [84]:
# Convertir Ingreso a int64
datos_filtrados["Ingreso"] = datos_filtrados["Ingreso"].astype("int64")

In [87]:
datos_filtrados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29079 entries, 0 to 57743
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Sexo       29079 non-null  int64
 1   Edad       29079 non-null  int64
 2   Ocupacion  29079 non-null  int64
 3   Ingreso    29079 non-null  int64
dtypes: int64(4)
memory usage: 1.1 MB


# Analisis